# MD Test - GPU GROMACS (Step 7-9)

**Purpose:** Test if GROMACS GPU acceleration works on Kaggle

**Steps tested:**
- Step 7: Add Ions
- Step 8: Energy Minimization
- Step 9: NVT Equilibration

**Accelerator:** GPU P100 or T4

---

## Step 1: Install GROMACS with CUDA GPU Support

This compiles GROMACS from source with CUDA enabled (~10-15 min)

In [ ]:
%%bash
set -e

echo "=== Checking GPU ==="
nvidia-smi

echo "\n=== Checking CUDA ==="
nvcc --version || echo "nvcc not in PATH, checking /usr/local/cuda"
ls /usr/local/cuda*/bin/nvcc 2>/dev/null || echo "No CUDA found"

echo "\n=== GPU Check Complete ==="

In [ ]:
%%bash
set -e

echo "=== Installing Build Dependencies ==="
apt-get update -qq
apt-get install -qq -y cmake build-essential libfftw3-dev

echo "\n=== Downloading GROMACS 2024.4 ==="
cd /kaggle/working
if [ ! -f gromacs-2024.4.tar.gz ]; then
    wget -q https://ftp.gromacs.org/gromacs/gromacs-2024.4.tar.gz
fi
tar -xzf gromacs-2024.4.tar.gz

echo "Download complete!"

In [ ]:
%%bash
set -e

echo "=== Compiling GROMACS with CUDA ==="
cd /kaggle/working/gromacs-2024.4
mkdir -p build && cd build

# Find CUDA path
CUDA_PATH=$(ls -d /usr/local/cuda* 2>/dev/null | head -1)
echo "Using CUDA at: $CUDA_PATH"

# Configure with CUDA
cmake .. \
    -DGMX_GPU=CUDA \
    -DCUDA_TOOLKIT_ROOT_DIR=$CUDA_PATH \
    -DGMX_BUILD_OWN_FFTW=OFF \
    -DGMX_DOUBLE=OFF \
    -DCMAKE_INSTALL_PREFIX=/usr/local/gromacs \
    2>&1 | tail -20

echo "\n=== Configuration complete, starting build... ==="

In [ ]:
%%bash
set -e

echo "=== Building GROMACS (this takes ~10 min) ==="
cd /kaggle/working/gromacs-2024.4/build

# Build with 4 parallel jobs
make -j4 2>&1 | tail -20

echo "\n=== Installing ==="
make install 2>&1 | tail -10

echo "\n=== GROMACS GPU Build Complete! ==="

In [ ]:
%%bash
set -e

# Source GROMACS environment
source /usr/local/gromacs/bin/GMXRC

echo "=== GROMACS Version ==="
gmx --version | head -20

echo "\n=== GPU Detection ==="
gmx --version | grep -i -A5 "GPU" || echo "Checking GPU support..."

echo "\n=== GROMACS GPU Ready! ==="

## Step 2: Setup Working Directory

Copy files from previous run dataset

In [ ]:
import os
import shutil
from pathlib import Path

CONFIG = {
    'complex_name': '264THM_PPARG',
    'temperature_k': 310,
    'timestep_fs': 2,
    'nvt_time_ps': 100,
}

WORK_DIR = Path(f"/kaggle/working/{CONFIG['complex_name']}")
TOPOL_DIR = WORK_DIR / 'topol'

# Create directories
for d in ['topol', 'em', 'nvt']:
    (WORK_DIR / d).mkdir(parents=True, exist_ok=True)

print(f'Working directory: {WORK_DIR}')

In [ ]:
# Copy files from dataset (adjust path if needed)
DATASET_DIR = Path('/kaggle/input/md-264thm-pparg-solvated')

if DATASET_DIR.exists():
    # Copy all topology files
    for f in DATASET_DIR.glob('**/*'):
        if f.is_file():
            rel_path = f.relative_to(DATASET_DIR)
            dest = TOPOL_DIR / rel_path.name
            shutil.copy(f, dest)
            print(f'Copied: {f.name}')
else:
    print(f'Dataset not found: {DATASET_DIR}')
    print('Please upload your solvated files as a Kaggle dataset first!')
    print('Required files: solvated.gro, topol.top, ligand.itp, posre.itp')

## Step 7: Add Ions (Neutralize System)

In [ ]:
# Create ions.mdp
ions_mdp = '''integrator = steep
emtol = 1000.0
emstep = 0.01
nsteps = 50000
nstlist = 10
cutoff-scheme = Verlet
coulombtype = cutoff
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz
'''
with open(TOPOL_DIR / 'ions.mdp', 'w') as f:
    f.write(ions_mdp)
print('ions.mdp created')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC

cd /kaggle/working/264THM_PPARG/topol

echo "=== Creating ions.tpr ==="
gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 2

echo "\n=== Adding Ions ==="
printf 'SOL\n' | gmx genion -s ions.tpr -o system.gro -p topol.top -pname NA -nname CL -neutral -conc 0.15

echo "\nIons: OK"

## Step 8: Energy Minimization (GPU Test)

In [ ]:
em_mdp = '''integrator = steep
emtol = 1000.0
emstep = 0.01
nsteps = 50000
nstlist = 10
cutoff-scheme = Verlet
coulombtype = PME
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz
'''
with open(WORK_DIR / 'em' / 'em.mdp', 'w') as f:
    f.write(em_mdp)
print('em.mdp created')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC

cd /kaggle/working/264THM_PPARG

echo "=== Preparing Energy Minimization ==="
gmx grompp -f em/em.mdp -c topol/system.gro -p topol/topol.top -o em/em.tpr -maxwarn 2

echo "\n=== Running Energy Minimization (GPU) ==="
echo "Watch for 'Using GPU' message!"
echo ""

# Run with verbose output to see GPU usage
gmx mdrun -v -deffnm em/em -nb gpu 2>&1 | head -50

echo "\n... (output truncated) ..."
gmx mdrun -v -deffnm em/em -nb gpu 2>&1 | tail -20

echo "\n=== ENERGY MINIMIZATION COMPLETE ==="

## Step 9: NVT Equilibration (GPU Test)

In [ ]:
dt = CONFIG['timestep_fs'] / 1000
nvt_steps = int(CONFIG['nvt_time_ps'] * 1000 / CONFIG['timestep_fs'])

nvt_mdp = f'''define = -DPOSRES
integrator = md
nsteps = {nvt_steps}
dt = {dt}
nstxout = 5000
nstvout = 5000
nstenergy = 5000
nstlog = 5000
continuation = no
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter = 1
lincs_order = 4
cutoff-scheme = Verlet
nstlist = 10
rcoulomb = 1.0
rvdw = 1.0
coulombtype = PME
pme_order = 4
fourierspacing = 0.16
tcoupl = V-rescale
tc-grps = Protein Non-Protein
tau_t = 0.1 0.1
ref_t = {CONFIG['temperature_k']} {CONFIG['temperature_k']}
pcoupl = no
pbc = xyz
DispCorr = EnerPres
gen_vel = yes
gen_temp = {CONFIG['temperature_k']}
gen_seed = -1
'''
with open(WORK_DIR / 'nvt' / 'nvt.mdp', 'w') as f:
    f.write(nvt_mdp)
print(f'NVT: {CONFIG["nvt_time_ps"]} ps, {nvt_steps} steps')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC

cd /kaggle/working/264THM_PPARG

echo "=== Preparing NVT ==="
gmx grompp -f nvt/nvt.mdp -c em/em.gro -r em/em.gro -p topol/topol.top -o nvt/nvt.tpr -maxwarn 2

echo "\n=== Running NVT Equilibration (GPU) ==="
echo "This should be MUCH faster with GPU!"
echo ""

# Run with GPU acceleration
time gmx mdrun -v -deffnm nvt/nvt -nb gpu -pme gpu -bonded gpu

echo "\n=== NVT EQUILIBRATION COMPLETE ==="

## Results Summary

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC

echo "=== Energy Minimization Results ==="
tail -20 /kaggle/working/264THM_PPARG/em/em.log 2>/dev/null || echo "EM log not found"

echo "\n=== NVT Results ==="
tail -30 /kaggle/working/264THM_PPARG/nvt/nvt.log 2>/dev/null || echo "NVT log not found"

echo "\n=== Performance Comparison ==="
echo "If GPU is working, you should see:"
echo "- Performance > 50 ns/day (vs ~5 ns/day on CPU)"
echo "- 'GPU' mentioned in the output"
echo "- NVT completing in ~2-5 minutes (vs ~30 min on CPU)"

In [ ]:
# List output files
import os
for root, dirs, files in os.walk(WORK_DIR):
    for f in files:
        path = os.path.join(root, f)
        size = os.path.getsize(path)
        print(f'{path}: {size/1024:.1f} KB')